In [1]:
!pip install -U ipywidgets
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
import os
current_directory = os.getcwd()
current_directory
sys.path.append(current_directory+'\\..\\fl_g13')

In [3]:
from fl_g13.modeling.test import test_model
from fl_g13.modeling.train import train_model
from fl_g13.config import RAW_DATA_DIR
from collections import OrderedDict
from typing import List


import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets, transforms
from collections import Counter

from fl_g13.base_experimentation import dataset_handler
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset


2025-04-15 16:36:24.531 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


# Load data

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])
cifar100_train = datasets.CIFAR100(root=RAW_DATA_DIR, train=True, download=True, transform=transform)

In [6]:
### train val split
train_dataset,val_dataset = dataset_handler.train_test_split(cifar100_train,train_ratio=0.8)

In [7]:
# I.I.D Sharding Split
## k client
k =10
clients_dataset_train= dataset_handler.iid_sharding(train_dataset,k)
clients_dataset_val= dataset_handler.iid_sharding(val_dataset,k)

In [8]:
dataset_handler.check_subset_distribution(clients_dataset_train[2])

Counter({63: 57,
         8: 53,
         2: 50,
         32: 50,
         33: 49,
         83: 49,
         9: 49,
         70: 48,
         40: 48,
         59: 47,
         20: 47,
         85: 47,
         38: 46,
         23: 46,
         37: 46,
         81: 45,
         89: 45,
         14: 45,
         34: 45,
         48: 45,
         45: 45,
         68: 44,
         76: 44,
         42: 44,
         6: 44,
         95: 44,
         0: 43,
         29: 43,
         30: 43,
         82: 43,
         47: 42,
         31: 42,
         73: 42,
         67: 42,
         4: 42,
         54: 42,
         62: 42,
         78: 42,
         19: 42,
         96: 41,
         92: 41,
         94: 41,
         11: 41,
         99: 41,
         7: 41,
         35: 40,
         84: 40,
         24: 40,
         41: 40,
         64: 39,
         44: 39,
         58: 39,
         46: 39,
         90: 39,
         80: 39,
         12: 39,
         36: 39,
         18: 39,
         69: 38,
    

# Update model parameters

In [9]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)



def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

# Define the ClientApp

In [10]:
from enum import Enum
import glob
from pathlib import Path

import torch
from loguru import logger
from tqdm import tqdm
import typer



app = typer.Typer()

class MODEL_DICTIONARY(Enum):
    EPOCH = 'epoch'
    MODEL_STATE_DICT = 'model_state_dict'
    OPTIMIZER_STATE_DICT = 'optimizer_state_dict'


def save_model(model, optimizer, checkpoint_dir, epoch=None, prefix_name="model"):
    os.makedirs(checkpoint_dir, exist_ok=True)
    if epoch is None:
        checkpoint_path = os.path.join(checkpoint_dir, f"{prefix_name}.pth")
    else:
        checkpoint_path = os.path.join(checkpoint_dir, f"{prefix_name}_epoch_{epoch}.pth")
    checkpoint_path = os.path.join(checkpoint_dir, checkpoint_path)

    torch.save({
        MODEL_DICTIONARY.EPOCH.value: epoch,
        MODEL_DICTIONARY.MODEL_STATE_DICT.value: model.state_dict(),
        MODEL_DICTIONARY.OPTIMIZER_STATE_DICT.value: optimizer.state_dict(),
    }, checkpoint_path)

    print(f"💾 Saved checkpoint at: {checkpoint_path}")


def load_or_create_model(checkpoint_dir=None, model=None, optimizer=None, lr=1e-4, weight_decay=0.04, device=None):
    if not checkpoint_dir and not model:
        raise ValueError("Either checkpoint_dir or model must be provided.")
    if optimizer is None:
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    if not checkpoint_dir:
        model.to(device)
        return model, optimizer, 1

    if not device:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if model is None:
        vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits16')
        model = vits16
        model.to(device)



    checkpoint_files = sorted(
        glob.glob(os.path.join(checkpoint_dir, "*.pth")),
        key=os.path.getmtime
    )

    if checkpoint_files:
        # Load the latest checkpoint
        latest_ckpt = checkpoint_files[-1]
        checkpoint = torch.load(latest_ckpt, map_location=device)
        model.load_state_dict(checkpoint[MODEL_DICTIONARY.MODEL_STATE_DICT.value])
        optimizer.load_state_dict(checkpoint[MODEL_DICTIONARY.OPTIMIZER_STATE_DICT.value])
        start_epoch = checkpoint[MODEL_DICTIONARY.EPOCH.value] + 1
        print(f"Loaded checkpoint from {latest_ckpt}, resuming at epoch {start_epoch}")
    else:
        start_epoch = 1
        print(f"No checkpoint found, initializing new model from scratch.")

    return model, optimizer, start_epoch


def _train(model, optimizer, dataloader, loss_fn, device, is_print=False):
    # TODD
    # print("Training...")
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    for batch, (X, y) in enumerate(dataloader):
        # print(f"Batch {batch+1}/{len(dataloader)}")
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(pred.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
        batch_acc = 100 * (predicted == y).sum().item() / y.size(0)
        if is_print:
            print(f"  ↳ Batch {batch + 1}/{len(dataloader)} | Loss: {loss.item():.4f} | Batch Acc: {batch_acc:.2f}%")

    training_loss = total_loss / len(dataloader)
    training_accuracy = 100 * correct / total
    print(f"Training Loss: {training_loss:.4f}, Training Accuracy: {training_accuracy:.2f}%")
    return training_loss, training_accuracy


def train_model(checkpoint_dir, dataloader, loss_fn=torch.nn.CrossEntropyLoss(),
                num_epochs=10, save_every=None, lr=1e-4, weight_decay=0.04,
                model=None, optimizer=None, device=None, print_batch=False):

    if not device:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    model, optimizer, start_epoch = load_or_create_model(
        checkpoint_dir=checkpoint_dir,
        model=model,
        optimizer=optimizer,
        lr=lr,
        weight_decay=weight_decay
    )
    for epoch in range(start_epoch, num_epochs + 1):
        avg_loss, training_accuracy = _train(model, optimizer, dataloader, loss_fn, device, print_batch)
        print(f"📘 Epoch [{epoch}/{num_epochs}] - Avg Loss: {avg_loss:.4f}, Accuracy: {training_accuracy:.2f}%")

        # 5. Save checkpoint
        if save_every and epoch % save_every == 0:
            save_model(model, optimizer, checkpoint_dir, epoch, prefix_name="dino_xcit")



In [11]:

def test_model(model, dataloader, loss_fn,device=None):
    # TODO
    model.eval()
    if not device:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    total_loss = 0.0
    correct = 0
    total = 0
    preds = []
    labels = []
    probs = []
    inputs = []
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)


            inputs.extend(X.cpu())
            total_loss += loss.item()
            original_pre = pred.cpu()
            prob, predicted = torch.max(pred.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()
            preds.extend(predicted.cpu())
            labels.extend(y.cpu())
            probs.extend(prob.cpu())

    test_loss = total_loss / len(dataloader)
    test_accuracy = 100 * correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
    preds = torch.tensor(preds)
    labels = torch.tensor(labels)
    probs = torch.tensor(probs)
    return preds, labels, probs, inputs, test_accuracy,test_loss

In [12]:
class FlowerClient(NumPyClient):
    def __init__(self, model, trainloader, valloader,loss_fn=torch.nn.CrossEntropyLoss(),optimizer=None):
        self.model = model
        self.trainloader = trainloader
        self.valloader = valloader
        self.loss_fn = loss_fn
        if not optimizer:
            optimizer = torch.optim.AdamW(self.model.parameters(), lr=1e-4, weight_decay=0.04)
        self.optimizer = optimizer

    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        train_model(checkpoint_dir=None,dataloader= self.trainloader, num_epochs=1,save_every=None,model=self.model,optimizer=self.optimizer,loss_fn=self.loss_fn,print_batch=False)
        return get_parameters(self.model), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        preds, labels, probs, inputs, test_accuracy,test_loss = test_model(self.model, self.valloader, self.loss_fn)
        return float(test_loss), len(self.valloader), {"accuracy": float(test_accuracy)}

## Tiny model

In [13]:
class TinyCNN(nn.Module):
    def __init__(self, num_classes=100):
        super(TinyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))     # -> [B, 16, 32, 32]
        x = F.max_pool2d(x, 2)        # -> [B, 16, 16, 16]
        x = F.relu(self.conv2(x))     # -> [B, 32, 16, 16]
        x = F.max_pool2d(x, 2)        # -> [B, 32, 8, 8]
        x = x.view(x.size(0), -1)     # -> [B, 32*8*8]
        x = self.fc1(x)               # -> [B, 100]
        return x

## create FlowerClient instances  

In [14]:
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = TinyCNN().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data partition
    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    
    trainloader = torch.utils.data.DataLoader(clients_dataset_train[partition_id])
    valloader = torch.utils.data.DataLoader(clients_dataset_val[partition_id])
    # Create a single Flower client representing a single organization
    # FlowerClient is a subclass of NumPyClient, so we need to call .to_client()
    # to convert it to a subclass of `flwr.client.Client`
    return FlowerClient(net, trainloader, valloader).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

# Define the Flower ServerApp

Using a built-in strategy Federated Averaging (FedAvg) of Flower to combine hyperparams in server-side

In [1]:
from flwr.common import FitRes, Scalar, Parameters
from flwr.server.client_proxy import ClientProxy
from typing import Union, Optional

net = TinyCNN().to(DEVICE)
class SaveModelStrategy(flwr.server.strategy.FedAvg):
    def aggregate_fit(
        self,
        server_round: int,
        results: list[tuple[flwr.server.client_proxy.ClientProxy, flwr.common.FitRes]],
        failures: list[Union[tuple[ClientProxy, FitRes], BaseException]],
    ) -> tuple[Optional[Parameters], dict[str, Scalar]]:
        """Aggregate model weights using weighted average and store checkpoint"""

        # Call aggregate_fit from base class (FedAvg) to aggregate parameters and metrics
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(
            server_round, results, failures
        )

        if aggregated_parameters is not None:
            print(f"Saving round {server_round} aggregated_parameters...")

            # Convert `Parameters` to `list[np.ndarray]`
            aggregated_ndarrays: list[np.ndarray] = flwr.common.parameters_to_ndarrays(
                aggregated_parameters
            )

            # Convert `list[np.ndarray]` to PyTorch `state_dict`
            params_dict = zip(net.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            net.load_state_dict(state_dict, strict=True)

            # Save the model to disk
            torch.save(net.state_dict(), f"model_round_{server_round}.pth")

        return aggregated_parameters, aggregated_metrics



NameError: name 'TinyCNN' is not defined

In [16]:
# Create FedAvg strategy
strategy = FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=10,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=10,  # Wait until all 10 clients are available
)

## Create instant of ServerApp

In [17]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use the settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    # Configure the server for 5 rounds of training
    config = ServerConfig(num_rounds=4)

    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

# Run the training


In [18]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [19]:
NUM_CLIENTS =10

In [20]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=4, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 4.5333, Training Accuracy: 2.55%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 4.5333, Accuracy: 2.55%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 4.5018, Training Accuracy: 2.70%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 4.5018, Accuracy: 2.70%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 4.5165, Training Accuracy: 3.17%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 4.5165, Accuracy: 3.17%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 4.5462, Training Accuracy: 2.08%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 4.5462, Accuracy: 2.08%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 4.5004, Training Accuracy: 3.02%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 4.5004, Accuracy: 3.02%
(ClientAppActor pid=3856)

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


(ClientAppActor pid=3856) Training Loss: 4.5430, Training Accuracy: 2.73%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 4.5430, Accuracy: 2.73%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Test Loss: 4.3545, Test Accuracy: 7.90%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Test Loss: 4.3486, Test Accuracy: 7.90%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Test Loss: 4.3284, Test Accuracy: 8.10%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Test Loss: 4.3489, Test Accuracy: 9.10%
(ClientAppActor pid=3856) Using device: cuda


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

(ClientAppActor pid=3856) Test Loss: 3.7608, Test Accuracy: 15.60%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 3.7935, Training Accuracy: 14.00%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 3.7935, Accuracy: 14.00%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 3.7961, Training Accuracy: 14.93%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 3.7961, Accuracy: 14.93%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 3.7637, Training Accuracy: 14.65%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 3.7637, Accuracy: 14.65%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 3.7654, Training Accuracy: 14.40%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 3.7654, Accuracy: 14.40%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Training Loss: 3.7610, Training Accuracy: 14.60%
(ClientAppActor pid=3856)

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


(ClientAppActor pid=3856) Training Loss: 3.7820, Training Accuracy: 14.40%
(ClientAppActor pid=3856) 📘 Epoch [1/1] - Avg Loss: 3.7820, Accuracy: 14.40%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Test Loss: 3.6768, Test Accuracy: 17.70%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Test Loss: 3.7401, Test Accuracy: 16.40%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Test Loss: 3.6217, Test Accuracy: 19.30%
(ClientAppActor pid=3856) Using device: cuda
(ClientAppActor pid=3856) Test Loss: 3.7083, Test Accuracy: 15.40%
(ClientAppActor pid=3856) Using device: cuda


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 4 round(s) in 512.72s
INFO :      	History (loss, distributed):
INFO :      		round 1: 4.343714739751816
INFO :      		round 2: 3.9405155551075937
INFO :      		round 3: 3.784399951606989
INFO :      		round 4: 3.694976790513098
INFO :      


(ClientAppActor pid=3856) Test Loss: 3.7280, Test Accuracy: 16.10%
